In [1]:
import numpy as np
from tqdm import tqdm
import os
import json
from torch.utils.data import Dataset, Sampler
import torch.utils.data as data

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torchvision
from torchvision import datasets,models,transforms
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import math
import os
import copy
import time
import sqlite3 as sql
import cv2
from tqdm import tqdm
from sam_implem import *
from operator import itemgetter

/home/mdelabrassinne/anaconda3/envs/envDeepLearning/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mdelabrassinne/anaconda3/envs/envDeepLearning/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/mdelabrassinne/anaconda3/envs/envDeepLearning/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
!pip install wandb -qU

In [3]:
import wandb

### Get features, labels and timecodes

In [4]:
features = {}
labels = {}
timeLabel = {}

features['train'] = []
features['validation'] = []
features['test'] = []

labels['train'] = []
labels['validation'] = []
labels['test'] = []

timeLabel['train'] = []
timeLabel['validation'] = []
timeLabel['test'] = []

path = '/scratch/users/mdelabrassinne/Database/'

nbMatch = 0

current = 'train'

for root, dirs, files in tqdm(os.walk(path)):
    #print(files)
    
    for file in files:    
        if file.endswith("Labels-v2.json"):
            p = os.path.dirname(os.path.join(root, file))
            dire = os.path.basename(p).replace('_', ' ')
            p = os.path.join(os.path.dirname(p), dire)
            
            for r, d, fs in os.walk(p):
                for f in fs:
                    if f.endswith("baidu_soccer_embeddings.npy"):
                        p_current = os.path.join(r, f)
                        features[current].append(p_current)
                        #print(p_current)

            
            jsonPath =(os.path.join(root, file))
            #print(jsonPath)
            
            #opening of the json
            
            f = open(jsonPath)
            d = json.load(f)
            #print(d)
            
            #print(jsonPath)
            
            try:
                #print('test')
                lab = [d['annotations'][i]['label'] for i in range(len(d['annotations'])) if d['annotations'][i]['gameTime'][0] ==  '1']         # 1fps, don't know if keep 
                time = [60 * int(d['annotations'][i]['gameTime'].split(':')[0][-2:]) + int(d['annotations'][i]['gameTime'].split(':')[1][:2]) for i in range(len(d['annotations'])) if d['annotations'][i]['gameTime'][0] ==  '1']
                #print([d['annotations'][i]['gameTime'] for i in range(len(d['annotations']))])
                
                #print(time)
                
                labels[current].append(lab)
                timeLabel[current].append(time)
                
                lab = [d['annotations'][i]['label'] for i in range(len(d['annotations'])) if d['annotations'][i]['gameTime'][0] ==  '2']         # 1fps, don't know if keep 
                time = [60 * int(d['annotations'][i]['gameTime'].split(':')[0][-2:]) + int(d['annotations'][i]['gameTime'].split(':')[1][:2]) for i in range(len(d['annotations'])) if d['annotations'][i]['gameTime'][0] ==  '2']
                
                #print(time)
                labels[current].append(lab)
                timeLabel[current].append(time)
                
            except:
                #print(d)
                lab = [d['predictions'][i]['label'] for i in range(len(d['predictions'])) if d['predictions'][i]['gameTime'][0] == '1']         # 1fps, don't know if keep 
                time = [60 * int(d['predictions'][i]['gameTime'].split(':')[0][-2:]) + int(d['predictions'][i]['gameTime'].split(':')[1][:2]) for i in range(len(d['predictions'])) if d['predictions'][i]['gameTime'][0] == '1']
                
                labels[current].append(lab)
                timeLabel[current].append(time)
                
                lab = [d['predictions'][i]['label'] for i in range(len(d['predictions'])) if d['predictions'][i]['gameTime'][0] == '2']         # 1fps, don't know if keep 
                time = [60 * int(d['predictions'][i]['gameTime'].split(':')[0][-2:]) + int(d['predictions'][i]['gameTime'].split(':')[1][:2]) for i in range(len(d['predictions'])) if d['predictions'][i]['gameTime'][0] == '2']
                
                labels[current].append(lab)
                timeLabel[current].append(time)
                
           
            nbMatch += 1     #one more match will be seen
            
            if nbMatch >= 450 and nbMatch < 475:
                current = 'validation'
            elif nbMatch >= 475:
                current = 'test'
            #print("end_1")
            
            
            

            
data_l = list(zip(features['train'], labels['train'], timeLabel['train']))
data_val = list(zip(features['validation'], labels['validation'], timeLabel['validation']))
data_test = list(zip(features['test'], labels['test'], timeLabel['test']))

98714it [01:03, 1547.13it/s]


In [5]:
print(len(data_l))
print(len(data_val))
print(len(data_test))

900
50
50


### Function for the transformation of the ground truths

In [6]:
nb = np.array([2566, 31810, 18918, 11674, 7896, 10521, 5820, 4836, 5256, 2200, 2047, 2839, 55, 2098, 1703, 173, 46])

In [7]:
cL = ['Kick-off', 'Ball out of play', 'Throw-in', 'Foul', 'Clearance', 'Indirect free-kick', 'Shots on target', 'Corner', 'Shots off target', 'Direct free-kick', 'Yellow card', 'Substitution', 'Red card', 'Offside', 'Goal', 'Penalty', 'Yellow->red card']

def getTrueCoef(label, timeCodes, T, nbClasses, r_c = 3, f = 1, r_d = 6):
    C = []      #confidence as a 2D np array instead of a list of tupes

    ## Init
    res_conf = np.zeros((T, nbClasses))         # initialization of the output C
    res_disp = np.zeros((T, nbClasses))         # initialization of the output D
    domain_disp = np.zeros((T, nbClasses))      # initialization of the domain for D ==> the domain has 1 when the value has to be computed for the loss on the displacement and 0 otherwise
    
    dictClasses = {}    # dictionnary containing the list wiht the timecodes of the actions for a given class

    ## Init of the dictionnary
    for c in range(nbClasses):
        dictClasses[c] = []

    ## Focus on the confidence part
    for i in range(len(timeCodes)):        #loop over obtained labels
        t = timeCodes[i]
        c = label[i]
        
        if(c in cL):                           # focus on wanted classes
            c = cL.index(label[i])

            # get the interval on which C contains 1 for a given class
            tmin = max(0, t - r_c * f)        # cannot be lower than 0
            tmax = min(t + r_c * f, T-1)      # cannot be larger than T-1
            inds = np.arange(tmin, tmax+1)    # select the timecodes between tmin and tmax
            res_conf[inds, c] = 1             # put 1 in C at the good position
            
            ## Update the dictionnary
            dictClasses[c].append(t)
            
        ## test for bad cases (don't occur)
        if(t > T):
            print('\n\naie => t = {} -- T = {}\n\n'.format(t, T))
            
        if(i >= 2700):
            print("possible issue ")
            
            
    ## Focus on the displacement part
    for c in range(nbClasses):
        ind = np.array(dictClasses[c])       # find the list of timecodes for a given action
        
        if(len(ind) > 0):
            for t in range(T):           # go through all frames

                ## Find the frame containing an occurence which is the closest (and get the corresponding index)
                try:
                    closestInd = np.argmin(np.abs(ind - t))         #take the one that is the most on the left if draw
                except:
                    print(ind)
                    print(t)
                    print(len(ind))

                ## Check if in the domain
                if(np.abs(t - ind[closestInd]) <= r_d * f):
                    res_disp[t, c] = t - ind[closestInd]      # put the displacement in the matrix
                    domain_disp[t, c] = 1                     # keep the value in the domain
            
        
    return res_conf, res_disp, domain_disp

test_C, test_D, dom = getTrueCoef(data_l[0][1],data_l[0][2], np.load(data_l[0][0]).shape[0], len(cL) )

print(np.where(test_C != 0))
print(np.where(test_D != 0))
print(np.where(dom != 0))

print(test_C.shape)
print(test_D.shape)
print(dom.shape)

(array([   0,    1,    2,    3,  130,  131,  132,  133,  134,  135,  136,
        146,  147,  148,  149,  150,  151,  152,  153,  154,  155,  156,
        157,  158,  159,  179,  180,  181,  182,  183,  184,  185,  243,
        244,  245,  246,  247,  248,  249,  255,  256,  257,  258,  259,
        260,  261,  267,  268,  268,  269,  269,  270,  270,  271,  271,
        272,  272,  273,  273,  274,  289,  290,  291,  292,  293,  294,
        295,  316,  317,  318,  319,  320,  321,  322,  344,  345,  346,
        347,  348,  349,  350,  392,  393,  394,  395,  396,  397,  398,
        418,  419,  420,  421,  422,  423,  424,  437,  438,  439,  440,
        441,  442,  443,  452,  453,  454,  455,  456,  457,  458,  459,
        460,  461,  462,  463,  464,  464,  465,  465,  466,  467,  468,
        469,  470,  536,  537,  538,  539,  540,  541,  542,  559,  560,
        561,  562,  563,  564,  565,  565,  566,  567,  568,  569,  570,
        571,  582,  583,  584,  584,  585,  585,  

### Dataset, dataloader

In [8]:
cL = ['Kick-off', 'Ball out of play', 'Throw-in', 'Foul', 'Clearance', 'Indirect free-kick', 'Shots on target', 'Corner', 'Shots off target', 'Direct free-kick', 'Yellow card', 'Substitution', 'Red card', 'Offside', 'Goal', 'Penalty', 'Yellow->red card']

class BaiduDataset(Dataset):    #modifiable
    """
    Class to prepare the data for a neural network in pyTorch. The database is composed of 3 tables:
    - Video: to save the video information
    - Sequence: to save the sequence information labelled with the soccer actions and record the sound of the sequence
    - Image: to save the image information of the sequence
    """
    def __init__(self, data_list, transform=None):
        self.data_list = data_list
        self.transform = transform
        self.seq_length = 112           # number of frames in each sequence
        self.nb_per_item = 10           # number of sequences per item selected
        self.border = 12
        
        ## dynamic dictionnary for classes
        d = {}
        for c in cL:
            d[c] = 1
        self.dictClasses = d
        
        
    def updateDict(self, labels, timecodes, startTime,endTime):
        
        indices = np.where(np.logical_and(timecodes>=startTime, timecodes<=endTime))[0]             #indices verifying the condition
        seenClasses = labels[indices]                                                         #classes occuring in the sequence   
        for classSeen in seenClasses:
            self.dictClasses[classSeen] += 1                                                  #adding occurence to the vector
            
        
    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        feat_path = self.data_list[idx][0]                 # path towards Baidu features
        label_list = np.array(self.data_list[idx][1])      # list of labels
        time_list = np.array(self.data_list[idx][2])       # list of timecodes

        features = np.load(feat_path)

        features_out_list = []
        coef_out_c = []
        coef_out_d = []
        dom_out_list = []
        inf_list = []

        for _ in range(self.nb_per_item):
            presentLabels = np.unique(label_list)                                        # only focus on current classes
                        
            weights = np.array(list(itemgetter(*presentLabels)(self.dictClasses)))       # get weights form the dictionnary for those classes
                        
            weights = weights / sum(weights)                                                      # normalize the associated probability vector
            complWeight = 1-weights                                                      # complementary of the probabilities  (more importance for unlikely events)
            complWeight /= sum(complWeight)                                              # normalize the new probabilities   
                        
            selectedClass = presentLabels[np.argmax(np.random.multinomial(1, complWeight, size=1))]         # select the class according to the weights
            
            events = np.where(label_list == selectedClass)[0]                            # consider the indices of the selected class
                        
            selectedEvent = events[np.random.randint(0, len(events))]                    # select the event of the class among the different possibilities (and only consider the index)
                        
            selectedTimestep = time_list[selectedEvent]
                                    
            
            selectedStart = np.random.randint(max(0, selectedTimestep - self.seq_length), min(features.shape[0] - self.seq_length+1, max(1, selectedTimestep)))
                        
            self.updateDict(label_list, time_list, selectedStart, selectedStart+112)
                        
            ind = np.array([True if (i >= selectedStart and i < selectedStart + self.seq_length) else False for i in range(features.shape[0])])

            coef_c, coef_d, dom = getTrueCoef(label_list, time_list, features.shape[0], len(cL) )

            features_out = torch.from_numpy(features[ind,:]).unsqueeze(0)
            coef_out_c.append(torch.from_numpy(coef_c[ind, :]).unsqueeze(0))
            coef_out_d.append(torch.from_numpy(coef_d[ind, :]).unsqueeze(0))
            dom_out_list.append(torch.from_numpy(dom[ind, :]).unsqueeze(0))
            features_out_list.append(features_out)

        return (torch.cat(features_out_list, dim=0), torch.cat(coef_out_c, dim=0), torch.cat(coef_out_d, dim=0), torch.cat(dom_out_list, dim=0), idx) 

### Model

In [9]:
MLP_KERNEL_SIZE = 1
MLP_FILTERS_1 = 256
MLP_FILTERS_2 = 64
CONTRACTION_RATIO = 2
PADDING_METHOD = 'same'
UPSAMPLE_MODE = 'nearest'
N_FEATURE_EXTRACTED = 8576
N_CLASSES = 17
SEQUENCE_SIZE = 112
BATCH_SIZE = 20
BATCH_PER_EPOCH = 1000

In [10]:
class MLP2(nn.Module):
    def __init__(self,n_features):
        super(MLP2,self).__init__()
        self.conv1 = nn.Conv1d(n_features,MLP_FILTERS_1,MLP_KERNEL_SIZE)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(MLP_FILTERS_1,MLP_FILTERS_2,MLP_KERNEL_SIZE)

    def forward(self,x):
        out = self.conv1(x)
        out = self.relu(out)
        out = self.conv2(out)
        return out


In [11]:
class BottleneckResBlockContract(nn.Module):
    # Takes as input a feature matrix of size:
    #  _____
    # |     |
    # |     |
    # |     |
    # |     | T
    # |     |
    # |_____|
    #    P
    # Performs a bottleneck Resnetblock:
    # BN -> ReLU -> conv(1,P) ->BN ->ReLU -> conv(3,P) -> BN -> ReLU -> conv(1,2P) -> + -> output (of size T,2P)
    # |                                                                               ^
    # L>    ->  ->  ->  ->  ->  ->  ->  -> conv(1,2P) ->  ->  ->  ->    ->  ->  ->  ->|  
    def __init__(self,in_features):
        super(BottleneckResBlockContract,self).__init__()
        self.bn1 = nn.BatchNorm1d(in_features)
        self.relu = nn.ReLU()
        self.conv1 = nn.Conv1d(in_features,in_features,1)
        self.bn2 = nn.BatchNorm1d(in_features)
        self.conv2 = nn.Conv1d(in_features,in_features,3,padding=PADDING_METHOD)
        self.bn3 = nn.BatchNorm1d(in_features)
        self.conv3 = nn.Conv1d(in_features,in_features*CONTRACTION_RATIO,1)
        self.conv_skip = nn.Conv1d(in_features,in_features*CONTRACTION_RATIO,1)

    def forward(self,x):
        y = self.bn1(x)
        y = self.relu(y)
        y = self.conv1(y)
        y = self.bn2(y)
        y = self.relu(y)
        y = self.conv2(y)
        y = self.bn3(y)
        y = self.relu(y)
        y = self.conv3(y)
        z = self.conv_skip(x)
        return y+z
    
class BottleneckResBlockExpand(nn.Module):
    # Takes as input a feature matrix of size:
    #  _____
    # |     |
    # |     |
    # |     |
    # |     | T
    # |     |
    # |_____|
    #    P
    # Performs a bottleneck Resnetblock:
    # BN -> ReLU -> conv(1,P/2) ->BN ->ReLU -> conv(3,P/2) -> BN -> ReLU -> conv(1,P/2) -> + -> output (of size T,P/2)
    # |                                                                               ^
    # L>    ->  ->  ->  ->  ->  ->  ->  -> conv(1,P/2) ->  ->  ->  ->    ->  ->  ->  ->|  
    def __init__(self,in_features):
        in_features = int(in_features)
        super(BottleneckResBlockExpand,self).__init__()
        self.bn1 = nn.BatchNorm1d(int(in_features))
        self.relu = nn.ReLU()
        self.conv1 = nn.Conv1d(in_features,int(in_features/CONTRACTION_RATIO),1)
        self.bn2 = nn.BatchNorm1d(int(in_features/CONTRACTION_RATIO))
        self.conv2 = nn.Conv1d(int(in_features/CONTRACTION_RATIO),int(in_features/CONTRACTION_RATIO),3,padding=PADDING_METHOD)
        self.bn3 = nn.BatchNorm1d(int(in_features/CONTRACTION_RATIO))
        self.conv3 = nn.Conv1d(int(in_features/CONTRACTION_RATIO),int(in_features/CONTRACTION_RATIO),1)
        self.conv_skip = nn.Conv1d(in_features,int(in_features/CONTRACTION_RATIO),1)

    def forward(self,x):
        y = self.bn1(x)
        y = self.relu(y)
        y = self.conv1(y)
        y = self.bn2(y)
        y = self.relu(y)
        y = self.conv2(y)
        y = self.bn3(y)
        y = self.relu(y)
        y = self.conv3(y)
        z = self.conv_skip(x)
        return y+z

class UpsamplingBlock(nn.Module):
# Takes as input a feature matrix of size (with ncol: nFeatures and nrow: nTimePoints) :
    #  _____
    # |     |
    # |     |
    # |     |
    # |     | T
    # |     |
    # |_____|
    #    P
# Upsample uses nearest neighbor to output a 2TxP matrix
# a Conv(2,P/2) is then applied with padding = same
    
    def __init__(self,in_feature):
        super(UpsamplingBlock,self).__init__()
        in_feature = int(in_feature)
        self.upsample = nn.Upsample(scale_factor=2,mode=UPSAMPLE_MODE)
        self.conv = nn.Conv1d(in_feature, int(in_feature/2),kernel_size=2, padding=PADDING_METHOD)
    
    def forward(self,x):
        out = self.upsample(x)
        out = self.conv(out)
        return out
    

class Unet1D(nn.Module):
    def __init__(self,in_feature):
        super(Unet1D,self).__init__()

        #LEFT PART
        self.maxpool = nn.MaxPool1d(2)

        self.contr1 = BottleneckResBlockContract(in_feature)
        #self.maxpool
        nb_f = in_feature*2
        self.contr2 = BottleneckResBlockContract(nb_f)
        #self.maxpool
        nb_f = nb_f*2
        self.contr3 = BottleneckResBlockContract(nb_f)
        #self.maxpool
        nb_f = nb_f*2
        #self.contr4 = BottleneckResBlockContract(nb_f)
        #self.maxpool
        #nb_f = nb_f*2
        ##############################################

        #DOWN PART
        
        self.contr5 = BottleneckResBlockContract(nb_f)
        nb_f = nb_f*2
        ##############################################

        #RIGHT PART

        #self.upsample4 = UpsamplingBlock(nb_f)
        #nb_f = nb_f/2
        #concat
        #nb_f = nb_f*2
        #self.expand4 = BottleneckResBlockExpand(nb_f)
        #nb_f = nb_f/2

        self.upsample3 = UpsamplingBlock(nb_f)
        nb_f = nb_f/2
        #concat
        nb_f = nb_f*2
        self.expand3 = BottleneckResBlockExpand(nb_f)
        nb_f = nb_f/2

        self.upsample2 = UpsamplingBlock(nb_f)
        nb_f = nb_f/2
        #concat
        nb_f = nb_f*2
        self.expand2 = BottleneckResBlockExpand(nb_f)
        nb_f = nb_f/2

        self.upsample1 = UpsamplingBlock(nb_f)
        nb_f = nb_f/2
        #concat
        nb_f = nb_f*2
        self.expand1 = BottleneckResBlockExpand(nb_f)
        nb_f = nb_f/2

        self.upsample0 = UpsamplingBlock(nb_f)
        nb_f = nb_f/2
        #concat
        nb_f = nb_f*2
        self.expand0 = BottleneckResBlockExpand(nb_f)
        nb_f = nb_f/2

    def forward(self,x):
        # X is of shape [nbatch,64,224]

        # LEFT PART
        l0 = x #[P,T]           #Output shape
        a = self.maxpool(l0)    #[P,T/2]
        l1 = self.contr1(a)     #[P*2,T/2]
        b = self.maxpool(l1)    #[P*2,T/4]
        l2 = self.contr2(b)     #[P*4,T/4]
        c = self.maxpool(l2)    #[P*4,T/8]
        l3 = self.contr3(c)     #[P*8,T/8]
        d = self.maxpool(l3)    #[P*8,T/16]
        #l4 = self.contr4(d)     #[P*16,T/16]
        #e = self.maxpool(l4)    #[P*16, T/32]

        # DOWN PART
        l5 = self.contr5(d)     #[P*32,T/32]

        # RIGHT PART
        #r4 = self.upsample4(l5)     #[P*16,T/16]
        #t4 = torch.cat((l4,r4),1)   #[P*32,T/16]
        #t4 = self.expand4(t4)       #[P*16,T/16]
        r3 = self.upsample3(l5)     #[P*8,T/8]
        t3 = torch.cat((l3,r3),1)   #[P*16,T/8]
        t3 = self.expand3(t3)       #[P*8,T/8]
        r2 = self.upsample2(t3)     #[P*4,T/4]
        t2 = torch.cat((l2,r2),1)   #[P*8,T/4]
        t2 = self.expand2(t2)       #[P*4,T/4]
        r1 = self.upsample1(t2)     #[P*2,T/2]
        t1 = torch.cat((l1,r1),1)   #[P*4,T/2]
        t1 = self.expand1(t1)       #[P*2,T/2]
        r0 = self.upsample0(t1)     #[P,T]
        t0 = torch.concat((l0,r0),1)#[P*2,T]
        t0 = self.expand0(t0)       #[P,T]

        return t0 # P features, T points

In [12]:
class PredictionHead(nn.Sequential):
    def __init__(self,n_features):
        super(PredictionHead,self).__init__()
        self.mlp = MLP2(n_features)
        self.unet = Unet1D(MLP_FILTERS_2)
        self.convHead = nn.Conv1d(MLP_FILTERS_2,N_CLASSES,kernel_size=3,padding=PADDING_METHOD)


In [13]:
class TemporalHead(nn.Sequential):
    def __init__(self,n_features):
        super(TemporalHead,self).__init__()   
        self.mlp = MLP2(n_features)
        self.unet = Unet1D(MLP_FILTERS_2)
        self.convHead = nn.Conv1d(MLP_FILTERS_2,N_CLASSES,kernel_size=3,padding=PADDING_METHOD)


### test loss

In [14]:
import torch.nn as nn

In [15]:
def huberLoss(pred, gTruth, treshold=0.5):
    res = pred - gTruth
    
    abs_res = torch.abs(res)
    quadratic_loss = 0.5 * torch.square(res)
    linear_loss = treshold * (abs_res - 0.5 * treshold)
    
    loss = torch.where(abs_res <= treshold, quadratic_loss, linear_loss)
    return torch.sum(loss)

In [16]:
def confLoss2(c_preds, c_truths):
    criterion = nn.BCEWithLogitsLoss(reduction='none')
    
    c_preds = c_preds.reshape(-1)
    c_truths = c_truths.reshape(-1)
    
    loss = criterion(c_preds, c_truths)
    total_loss = loss.sum()
    
    return total_loss

def dispLoss2(d_preds, d_truths, dom):
    #optimized version 
    
    dom = dom.bool()   # convert the domain mask to boolean tensor
    dp = torch.masked_select(d_preds, dom)   # select the predicted values where the mask is True
    dt = torch.masked_select(d_truths, dom)  # select the ground truth values where the mask is True
    loss = huberLoss(dp, dt)
    return loss

### Data loader

In [17]:
nbPerBatch = 20
nbPerSequence = 10
nbcL = len(cL)


In [18]:
dataLearning = BaiduDataset(data_l)
loaderLearning = data.DataLoader(dataLearning, batch_size = nbPerBatch, shuffle= True, num_workers = 1)

### Confidence

In [19]:
device = 'cuda'
model = PredictionHead(N_FEATURE_EXTRACTED)

In [20]:
wandb.init(project = 'test_1', entity = "uliege_action_spotting_2022_2023_context", name = "test_new_dataset")
wandb.watch(model, confLoss2, log="all", log_freq=10)
#torch.onnx.export(model, images, "model_confidence.onnx")
#wandb.save("model_confidence.onnx")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jepi1202 (uliege_action_spotting_2022_2023_context). Use `wandb login --relogin` to force relogin


[]

In [ ]:
## Define loss
#loss_confidence = confLoss2

## Define optimizer
base_optimizer = torch.optim.Adam
optimizer = SAM(model.parameters(), base_optimizer, lr=0.001)

## put model on GPU
model.to(device)

## Train the model
num_epochs = 1000
for epoch in range(num_epochs):
    for features, coef_conf, coef_disp, dom, _ in tqdm(loaderLearning):
        
        features = features.view(nbPerBatch*nbPerSequence, 112, 8576)
        coef_conf = coef_conf.view(nbPerBatch*nbPerSequence, 112, nbcL)
        coef_disp = coef_disp.view(nbPerBatch*nbPerSequence, 112, nbcL)
        dom = dom.view(nbPerBatch*nbPerSequence, 112, nbcL)
        
        
        features, coef_conf, coef_disp, dom  = features.to(device), coef_conf.to(device), coef_disp.to(device), dom.to(device)  #check si obligé de tout mettre

        # Transpose for the way the NN is built
        dom = dom.transpose(1,2)
        coef_conf = coef_conf.transpose(1,2)
        coef_disp = coef_disp.transpose(1,2)
        
        T = features.shape[1]
        features = features.transpose(1, 2)
        
        print(features.shape)
        # Forward pass
        outputs = model(features)
        
        # Apply the loss
        loss = confLoss2(outputs, coef_conf)
        
        wandb.log({"epoch": epoch, "loss_confidence": loss})
        
        
        # Backward and optimize
        
        # first step
        #optimizer.zero_grad()
        loss.backward()
        optimizer.first_step(zero_grad=True)
        #optimizer.step(closure)
        
        # second step
        
        confLoss2(model(features), coef_conf).backward()
        optimizer.second_step(zero_grad=True)

        #val = val.to("cpu")
        #images = images.to("cpu")
        
        print("test")

    # Print training loss after every epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    
    if((epoch+1)%10 == 0):
        #torch.save(model.state_dict(), "confidence_head_test0.pt")

  0%|          | 0/45 [00:00<?, ?it/s]

torch.Size([200, 8576, 112])


/home/mdelabrassinne/anaconda3/envs/envDeepLearning/lib/python3.9/site-packages/torch/nn/modules/conv.py:309: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /opt/conda/conda-bld/pytorch_1670525551200/work/aten/src/ATen/native/Convolution.cpp:895.)
  return F.conv1d(input, weight, bias, self.stride,
  2%|▏         | 1/45 [00:49<36:21, 49.57s/it]

test
torch.Size([200, 8576, 112])


  4%|▍         | 2/45 [01:32<32:42, 45.63s/it]

...

test
torch.Size([200, 8576, 112])


 13%|█▎        | 6/45 [05:05<32:44, 50.38s/it]

test
torch.Size([200, 8576, 112])


 16%|█▌        | 7/45 [05:52<31:19, 49.46s/it]

test


### Displacement

In [30]:
#torch.save(model.state_dict(), "confidence_head_test1.pt")

In [19]:
model = TemporalHead(N_FEATURE_EXTRACTED)
device = 'cuda'

wandb.init(project = 'test_1', entity = "uliege_action_spotting_2022_2023_context", name = "test_v1_disp")
wandb.watch(model, dispLoss2, log="all", log_freq=10)
#torch.onnx.export(model, images, "model_confidence.onnx")
#wandb.save("model_confidence.onnx")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jepi1202 (uliege_action_spotting_2022_2023_context). Use `wandb login --relogin` to force relogin


[]

In [ ]:

## Define loss
#loss_confidence = confLoss2

## Define optimizer
base_optimizer = torch.optim.Adam
optimizer = SAM(model.parameters(), base_optimizer, lr=0.001)

## put model on GPU
model.to(device)

## Train the model
num_epochs = 1000
for epoch in range(num_epochs):
    for features, coef_conf, coef_disp, dom, _ in tqdm(loaderLearning):
        features, coef_conf, coef_disp, dom  = features.to(device), coef_conf.to(device), coef_disp.to(device), dom.to(device)  #check si obligé de tout mettre
        
        features = features.view(nbPerBatch*nbPerSequence, 112, 8576)
        coef_conf = coef_conf.view(nbPerBatch*nbPerSequence, 112, nbcL)
        coef_disp = coef_disp.view(nbPerBatch*nbPerSequence, 112, nbcL)
        dom = dom.view(nbPerBatch*nbPerSequence, 112, nbcL)
        
        # Transpose for the way the NN is built
        dom = dom.transpose(1,2)
        coef_conf = coef_conf.transpose(1,2)
        coef_disp = coef_disp.transpose(1,2)
        
        T = features.shape[1]
        features = features.transpose(1, 2)
        
        
        # Forward pass
        outputs = model(features)
        
        # Compute the loss
        loss = dispLoss2(outputs, coef_disp, dom)
        
        wandb.log({"epoch": epoch, "loss_disp": loss})
        
        
        # Backward and optimize
        
        
         # first step
        #optimizer.zero_grad()
        loss.backward()
        optimizer.first_step(zero_grad=True)
        #optimizer.step(closure)
        
        # second step
        
        dispLoss2(model(features), coef_disp, dom).backward()
        optimizer.second_step(zero_grad=True)

    # Print training loss after every epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    
    if((epoch+1)%5 == 0):
        torch.save(model.state_dict(), "disp_head_test0.pt")
        

torch.save(model.state_dict(), "disp_head_test0_last.pt")

  0%|          | 0/45 [00:00<?, ?it/s]/home/mdelabrassinne/anaconda3/envs/envDeepLearning/lib/python3.9/site-packages/torch/nn/modules/conv.py:309: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /opt/conda/conda-bld/pytorch_1670525551200/work/aten/src/ATen/native/Convolution.cpp:895.)
  return F.conv1d(input, weight, bias, self.stride,
100%|██████████| 45/45 [36:30<00:00, 48.68s/it]


Epoch [1/1000], Loss: 4827.4772


100%|██████████| 45/45 [36:38<00:00, 48.86s/it]


Epoch [2/1000], Loss: 4710.2542


100%|██████████| 45/45 [36:15<00:00, 48.35s/it]


Epoch [3/1000], Loss: 4174.1290


100%|██████████| 45/45 [37:27<00:00, 49.94s/it]


Epoch [4/1000], Loss: 4470.4191


 33%|███▎      | 15/45 [12:13<24:19, 48.64s/it]

In [27]:
torch.save(model.state_dict(), "disp_head_test0.pt")

### Training

In [60]:
config = dict(
    epochs=30,
    classes=17,
    learning_rate=0.0001,
    loaderLearning = loaderLearning,
    loaderValidation = loaderValidation,
    loaderTest = loaderTest)

NameError: name 'loaderValidation' is not defined

In [ ]:
def make():
    # Make the model
    mod = model.to(device)                                     #model variable globale

    # Make the loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, threshold  = .01, threshold_mode = "abs", mode='max', factor=0.2, patience=5)
    
    return criterion, optimizer, scheduler

In [ ]:
def train_log(loss_conf, loss_disp, example_ct, epoch):
    # Where the magic happens
    wandb.log({"epoch": epoch, "loss_confidence": loss_conf}, step=example_ct)
    wandb.log({"epoch": epoch, "loss_disp": loss_disp}, step=example_ct)
    print(f"Loss after {str(example_ct).zfill(5)} examples: {loss:.3f}")

In [ ]:
def test():
    model.eval()

    # Run the model on some test examples
    with torch.no_grad():
        correct, total = 0, 0
        correct_class = [0 for i in range(n_classes)]
        total_class = [0 for i in range(n_classes)]
        for features, coef_conf, coef_disp, dom, _ in loaderValidation:
           
            features, coef_conf, coef_disp, dom  = features.to(device), coef_conf.to(device), coef_disp.to(device), dom.to(device)  #check si obligé de tout mettre
        
            outputs = model(features)
            _, predicted = torch.max(outputs.data, 1)      # marche peut-être pas ...
            
            # COMPUTE THE PREDICTIONS FOR A MEASURE OF THE ACCURACY
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            # Compute accuracy per class
            for i in range(len(labels)):
                label = labels[i]
                pred = predicted[i]
                total_class[label] += 1
                if label == pred:
                    correct_class[label] += 1
            torch.cuda.empty_cache()

        
        test_acc = 0.
        for i in range(n_classes):
            test_acc += correct_class[i] / total_class[i]
            wandb.log({f"Accuracy of {c[i]}": correct_class[i] / total_class[i]})
        test_acc /= n_classes
        
        print(f"Accuracy of the model on the {total} " +
              f"test images: {test_acc:%}")
        
        wandb.log({"test_accuracy": test_acc})

        
        
    # Save the model in the exchangeable ONNX format
    torch.onnx.export(model, images, "model.onnx")
    wandb.save("model.onnx")
    return correct / total

In [ ]:
def train(criterion, optimizer, scheduler, test_set = True):
    # Tell wandb to watch what the model gets up to: gradients, weights, and more!
    wandb.watch(model, criterion, log="all", log_freq=10)

    # Run training and track with wandb
    total_batches = len(loaderLearning) * epochs
    example_ct = 0  # number of examples seen
    batch_ct = 0
        
    for epoch in tqdm(range(epochs)):
        for features, coef_conf, coef_disp, dom, _ in loaderLearning:
            
            #images = images.to(device)
            
            loss = train_batch(features, coef_conf, coef_disp, dom, model, optimizer, criterion)
            #example_ct +=  len(images)
            #batch_ct += 1

            # Report metrics every 25th batch
            if ((batch_ct + 1) % 25) == 0:
                train_log(loss_conf, loss_disp, example_ct, epoch)
                
            torch.cuda.empty_cache()

        if test_set:
            acc = test()
            scheduler.step(acc)
            


def train_batch(features, coef_conf, coef_disp, dom, model, optimizer, criterion):
    
    features, coef_conf, coef_disp, dom  = features.to(device), coef_conf.to(device), coef_disp.to(device), dom.to(device)  #check si obligé de tout mettre
        
    # Forward pass ➡
    outputs = model(images)
    loss = criterion(outputs, labels)
    
    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss

In [ ]:
config = dict(
    epochs=30,
    classes=6,
    learning_rate=0.0001,
    loaderLearning = loaderLearning,
    loaderValidation = loaderValidation,
    loaderTest = loaderTest)

In [ ]:
def model_pipeline(config=None, project_name="action_spotting_context"):

    # tell wandb to get started
    with wandb.init(config=config):
      # access all HPs through wandb.config, so logging matches execution!
      config = wandb.config

      # data, and optimization problem
      criterion, optimizer, scheduler  = make()  
      # and use them to train the model
      train(criterion, optimizer, scheduler)

    return model

In [ ]:
sweep_config = {
    "name": "action_spotting",
    "method": "random",
    "metric": {"name": "test_accuracy", "goal": "maximize"},
    "parameters":
    {
        "learning_rate": {"values": [0.01]},
        "batch_size": {"values": [100]},
    }
}
# sweep_id = wandb.sweep(sweep_config, project="action_spotting")

wandb.login()